In [1]:
# Importando as bibliotecas
import sympy as sp
from sympy.physics.vector import init_vprinting

init_vprinting(use_latex=True) #Printar em formatação Latex
t = sp.symbols('t') #Definindo t simbólico

#Definindo a matriz com os parâmetros de Denavid-Hatenberg
def dh_matrix(theta, d, a, alpha):
    c_t = sp.cos(theta)
    s_t = sp.sin(theta)
    c_a = sp.cos(alpha)
    s_a = sp.sin(alpha)
    A = sp.Matrix([
        [c_t, -s_t*c_a,  s_t*s_a, a*c_t],
        [s_t,  c_t*c_a, -c_t*s_a, a*s_t],
        [  0,      s_a,      c_a,     d],
        [  0,        0,        0,     1]
    ])
    return A


#Substitui valores muito pequenos (abaixo de 10^-12) por 0 em uma expressão SymPy.
TOLERANCIA = 1e-12

def chop_expr(expr, tolerance=TOLERANCIA):
    if hasattr(expr, 'atoms'):
        floats = expr.atoms(sp.Float)
        subs_dict = {f: 0 for f in floats if abs(f) < tolerance}
        return sp.simplify(expr.xreplace(subs_dict)) # Simplifica expressões algébricas
    return expr # Retorna o objeto se não for uma expressão sympy

# DEFINIÇÃO SIMBÓLICA DO MANIPULADOR (P-R-R-R)
print("="*70)
print("              DEFINIÇÃO DO MANIPULADOR")
print("="*70)

# Define as variáveis de junta como funções do tempo
q1_t = sp.Function('d1')(t)
q2_t = sp.Function('theta3')(t)
q3_t = sp.Function('theta4')(t)
q4_t = sp.Function('theta5')(t)

# Velocidades
q1d = q1_t.diff(t)
q2d = q2_t.diff(t)
q3d = q3_t.diff(t)
q4d = q4_t.diff(t)

# Acelerações
q1dd = q1_t.diff(t, 2)
q2dd = q2_t.diff(t, 2)
q3dd = q3_t.diff(t, 2)
q4dd = q4_t.diff(t, 2)

print("Variáveis de junta (q):\n")
display(sp.Matrix([q1_t, q2_t, q3_t, q4_t]))

# Definindo Pi
pi = sp.pi

# Calculando as matrizes de transformação com base no nosso manipulador

# J1 = 0, d1, 0, 0
A1_0 = dh_matrix(0, q1_t, 0, 0) # Matriz de 1 para 0

# J2 = 0, 0, 0.2, 0  (Elo Fixo "F")
A2_1 = dh_matrix(0, 0, 0.2, 0) # Matriz de 2 para 1

# J3 = tetha3, 0, 0, pi/2
A3_2 = dh_matrix(q2_t, 0, 0, pi/2) # Matriz de 3 para 2

# J4 = tetha4, 0.8, 0, pi/2
A4_3 = dh_matrix(q3_t, 0.8, 0, pi/2) # Matriz de 4 para 3

# J5 = tetha5, 0, 0.6, 0
A5_4 = dh_matrix(q4_t, 0, 0.6, 0) # Matriz de 5 para 4


# Cálculo da cinemática direta (Apenas para comparação)

print("\n" + "="*70)
print("                   CINEMÁTICA DIRETA")
print("="*70)

# Matrizes de transformação acumulativas
A2_0 = sp.simplify(A1_0 * A2_1) # Matriz de 2 para 0
A3_0 = sp.simplify(A2_0 * A3_2) # Matriz de 3 para 0
A4_0 = sp.simplify(A3_0 * A4_3) # Matriz de 4 para 0
A5_0 = sp.simplify(A4_0 * A5_4) # Matriz de 5 para 0

print("\nMatriz de Transformação Final (A5_0):\n")
display(A5_0)

# Posição do efetuador final (x, y, z)
P = A5_0[:3, 3]
print("\nVetor de Posição do Efetuador Final (P):\n")
display(P)


# Jacobiana Geométrica (6x4) (Apenas para comparação)

print("\n" + "="*70)
print("                  JACOBIANA GEOMÉTRICA (J)")
print("="*70 + "\n")

# Posições das origens e eixos
b0 = sp.Matrix([0, 0, 1])
R0 = sp.Matrix([0, 0, 0])

b1 = A1_0[:3, 2]; R1 = A1_0[:3, 3]
b2 = A2_0[:3, 2]; R2 = A2_0[:3, 3]
b3 = A3_0[:3, 2]; R3 = A3_0[:3, 3]
b4 = A4_0[:3, 2]; R4 = A4_0[:3, 3]

# Coluna 1 (Velocidade Linear e Anguladar de d1)
Jv1 = b0
Jw1 = sp.Matrix([0, 0, 0])
J1 = Jv1.col_join(Jw1) #Une as colunas

# Coluna 2 (Velocidade Linear e Anguladar de tetha3)
Jv2 = b2.cross(P - R2) #cross = produto vetorial
Jw2 = b2
J2 = Jv2.col_join(Jw2)

# Coluna 3 (Velocidade Linear e Anguladar de tetha4)
Jv3 = b3.cross(P - R3)
Jw3 = b3
J3 = Jv3.col_join(Jw3)

# Coluna 4 (Velocidade Linear e Anguladar de tetha5)
Jv4 = b4.cross(P - R4)
Jw4 = b4
J4 = Jv4.col_join(Jw4)

# Jacobiana Completa (6x4)
J = J1.row_join(J2).row_join(J3).row_join(J4)
J = sp.simplify(J)

display(J)

# DINÂMICA (MÉTODO DE EULER-LAGRANGE)
print("="*70)
print("                          DINÂMICA")
print("="*70)

# Parâmetros

g = 9.81 # Gravidade (m/s²)
rho=1000 # Densidade da água (Kg/m³)
m1, m2, m3, m4, m5 = [3.567, 0.803, 0.944, 2.349, 0.805] # Massa de cada elo (Kg)

#Vetores da origem do elo ao centro de massa
rc1x, rc1y, rc1z = 0,0,q1_t
rc2x, rc2y, rc2z = -0.1,0,0
rc3x, rc3y, rc3z = 0,0,-0.1
rc4x, rc4y, rc4z = 0,0,-0.4
rc5x, rc5y, rc5z = -0.3,0,0

# Transformando em matriz
rc1 = sp.Matrix([rc1x, rc1y, rc1z])
rc2 = sp.Matrix([rc2x, rc2y, rc2z])
rc3 = sp.Matrix([rc3x, rc3y, rc3z])
rc4 = sp.Matrix([rc4x, rc4y, rc4z])
rc5 = sp.Matrix([rc5x, rc5y, rc5z])

# Inércias na direção Ixx,Iyy e Izz de cada elo
Ixx1, Iyy1, Izz1 = 0.006319722, 0.029737395, 0.030119853 #Kg.m²
Ixx2, Iyy2, Izz2 = 0.000309233, 0.011100042, 0.011087581 #Kg.m²
Ixx3, Iyy3, Izz3 = 0.000416361, 0.013593867, 0.013516278 #Kg.m²
Ixx4, Iyy4, Izz4 = 0.273580729, 0.273362637, 0.001133000 #Kg.m²
Ixx5, Iyy5, Izz5 = 0.011110232, 0.000304593, 0.011085298 #Kg.m²

# Transformando em uma tensor de inércia diagonal (Assumindo Ixy,Ixz e Iyz = 0)
I1 = sp.diag(Ixx1, Iyy1, Izz1)
I2 = sp.diag(Ixx2, Iyy2, Izz2)
I3 = sp.diag(Ixx3, Iyy3, Izz3)
I4 = sp.diag(Ixx4, Iyy4, Izz4)
I5 = sp.diag(Ixx5, Iyy5, Izz5)

# Volume de cada elo (m³)
V1= 1500000.000E-9
V2= 378250.372E-9
V3= 402600.495E-9
V4= 1129109.758E-9
V5= 1077572.229E-9

# Cálculo da Energia Potencial (P)

g_vec = sp.Matrix([0, -g, 0]) # Vetor gravidade em Y negativo

# Posições dos Centros de Massa com relação a origem
Pc1 = R1 + A1_0[:3, :3] * rc1
Pc2 = R2 + A2_0[:3, :3] * rc2 # Elo fixo
Pc3 = R3 + A3_0[:3, :3] * rc3
Pc4 = R4 + A4_0[:3, :3] * rc4
Pc5 = P + A5_0[:3, :3] * rc5

# Potencial de Gravidade
P_grav = -m1 * g_vec.T * Pc1 - m2 * g_vec.T * Pc2 - m3 * g_vec.T * Pc3 - m4 * g_vec.T * Pc4 - m5 * g_vec.T * Pc5

# Potencial de Empuxo (Aproximação matemática)
Pe1=-rho*V1*g_vec.T
Pe2=-rho*V2*g_vec.T
Pe3=-rho*V3*g_vec.T
Pe4=-rho*V4*g_vec.T
Pe5=-rho*V5*g_vec.T
Pe = Pe1*Pc1 + Pe2*Pc2 + Pe3*Pc3 + Pe4*Pc4 + Pe5*Pc5

# P_total = P_grav + P_emp = (+m*g*z) + (-rho*V*g*z)
P_total = P_grav - Pe
P_total = P_total[0] # Converte de Matriz 1x1 para escalar

# Cálculo da Energia Cinética (K)

# Velocidades Angulares de cada elo
w1 = sp.Matrix([0, 0, 0])      # Elo 1 prismático, portanto w1 é 0
w2 = w1        # Elo 2 é fixo ao 1 (w2 também é 0)
w3 = w2 + b2*q2d # Rotação da Junta 3
w4 = w3 + b3*q3d # Rotação da Junta 4
w5 = w4 + b4*q4d # Rotação da Junta 5

# Velocidades Lineares dos centros de massa de cada elo
Vc1 = Pc1.diff(t)
Vc2 = Pc2.diff(t)
Vc3 = Pc3.diff(t)
Vc4 = Pc4.diff(t)
Vc5 = Pc5.diff(t)

# Energias Cinéticas
K1 = 0.5 * m1 * (Vc1.T * Vc1)[0] + 0.5 * (w1.T * A1_0[:3,:3] * I1 * A1_0[:3,:3].T * w1)[0]
K2 = 0.5 * m2 * (Vc2.T * Vc2)[0] + 0.5 * (w2.T * A2_0[:3,:3] * I2 * A2_0[:3,:3].T * w2)[0]
K3 = 0.5 * m3 * (Vc3.T * Vc3)[0] + 0.5 * (w3.T * A3_0[:3,:3] * I3 * A3_0[:3,:3].T * w3)[0]
K4 = 0.5 * m4 * (Vc4.T * Vc4)[0] + 0.5 * (w4.T * A4_0[:3,:3] * I4 * A4_0[:3,:3].T * w4)[0]
K5 = 0.5 * m5 * (Vc5.T * Vc5)[0] + 0.5 * (w5.T * A5_0[:3,:3] * I5 * A5_0[:3,:3].T * w5)[0]

K_total = K1 + K2 + K3 + K4 + K5 # Energia Cinética total

#Cálculo do Lagrangiano (L)
L = K_total - P_total

# Cálculo das matrizes M, C, e G

# Definir vetores de estado
q = sp.Matrix([q1_t, q2_t, q3_t, q4_t]) # Vetor posições das juntas
qd = sp.Matrix([q1d, q2d, q3d, q4d]) # Vetor velocidade das juntas
qdd = sp.Matrix([q1dd, q2dd, q3dd, q4dd]) # Vetor aceleração das juntas

try:
    #Calculaando o vetor de Gravidade/Empuxo (G(q) = d(P_total) / dq)
    G_vec = sp.Matrix([P_total.diff(qi) for qi in q])
    G_vec = sp.simplify(G_vec)
    G_vec = G_vec.applyfunc(chop_expr)

    # Calcular o Vetor Lagrangiano Completo (Tau_L = d/dt(dL/d(qd)) - dL/dq)
    Tau_L_list = []
    for i in range(4): # Para cada junta q_i
        dL_dqd = L.diff(qd[i])       # Derivada em relação à velocidade
        d_dt_dL_dqd = dL_dqd.diff(t) # Derivada no tempo (Regra da Cadeia)
        dL_dq = L.diff(q[i])         # Derivada em relação à posição

        # Tau_L_i = d/dt(dL/d(qd_i)) - dL/dq_i
        Tau_L_list.append(d_dt_dL_dqd - dL_dq)

    # Este é o vetor completo M(q)q_dd + C(q,qd) + G(q)
    Tau_L_vetor_sem_simplificar = sp.Matrix(Tau_L_list)

    #Calculando a Matriz de Inércia (M(q) = d(Tau_L) / d(q_dd))
    M_mat = Tau_L_vetor_sem_simplificar.jacobian(qdd)
    M_mat = sp.simplify(M_mat) # Simplifica a matriz final
    M_mat = M_mat.applyfunc(chop_expr)

    #Calcular Vetor de Coriolis/Centrípeta (C(q, qd)= Tau_L(com qdd=0) - G )

    # Substitui todas as acelerações por 0 em Tau_L
    C_vetor_sem_simplificar = Tau_L_vetor_sem_simplificar.subs([(qdd[i], 0) for i in range(4)])

    # Subtrai o vetor G que já calculamos
    C_vec = C_vetor_sem_simplificar - G_vec
    C_vec = sp.simplify(C_vec) # Simplifica o vetor final
    C_vec = C_vec.applyfunc(chop_expr)

    # Exibindo os resultados das matrizes
    print("\n" + "="*70)
    print("     EQUAÇÃO DINÂMICA FINAL: M(q)q_dd + C(q,qd) + G(q) = Tau")
    print("="*70)

    print("\nMatriz de Inércia M(q) [4x4]:\n")
    display(M_mat)

    print("\nVetor de Coriolis/Centrípeta C(q, qd) [4x1]:\n")
    display(C_vec)

    print("\nVetor de Gravidade/Empuxo G(q) [4x1]:\n")
    display(G_vec)


except Exception as e:
    print(f"\nOcorreu um erro.")

              DEFINIÇÃO DO MANIPULADOR
Variáveis de junta (q):



⎡d₁⎤
⎢  ⎥
⎢θ₃⎥
⎢  ⎥
⎢θ₄⎥
⎢  ⎥
⎣θ₅⎦


                   CINEMÁTICA DIRETA

Matriz de Transformação Final (A5_0):



⎡sin(θ₃)⋅sin(θ₅) + cos(θ₃)⋅cos(θ₄)⋅cos(θ₅)  sin(θ₃)⋅cos(θ₅) - sin(θ₅)⋅cos(θ₃)⋅ ↪
⎢                                                                              ↪
⎢sin(θ₃)⋅cos(θ₄)⋅cos(θ₅) - sin(θ₅)⋅cos(θ₃)  -sin(θ₃)⋅sin(θ₅)⋅cos(θ₄) - cos(θ₃) ↪
⎢                                                                              ↪
⎢             sin(θ₄)⋅cos(θ₅)                            -sin(θ₄)⋅sin(θ₅)      ↪
⎢                                                                              ↪
⎣                    0                                          0              ↪

↪ cos(θ₄)   sin(θ₄)⋅cos(θ₃)  0.6⋅sin(θ₃)⋅sin(θ₅) + 0.8⋅sin(θ₃) + 0.6⋅cos(θ₃)⋅c ↪
↪                                                                              ↪
↪ ⋅cos(θ₅)  sin(θ₃)⋅sin(θ₄)     0.6⋅sin(θ₃)⋅cos(θ₄)⋅cos(θ₅) - 0.6⋅sin(θ₅)⋅cos( ↪
↪                                                                              ↪
↪              -cos(θ₄)                            d₁ + 0.6⋅sin(θ₄)⋅cos(θ₅)    ↪
↪                          


Vetor de Posição do Efetuador Final (P):



...

⎡                 19.169                                                       ↪
⎢                                                                              ↪
⎢                                                                            2 ↪
⎢                   0                                         0.061644361⋅sin  ↪
⎢                                                                              ↪
⎢                                                                              ↪
⎢-0.9396⋅sin(θ₄) + 0.2415⋅cos(θ₄)⋅cos(θ₅)  -0.0966⋅sin(θ₄ - θ₅) - 0.0966⋅sin(θ ↪
⎢                                                                              ↪
⎣        -0.2415⋅sin(θ₄)⋅sin(θ₅)                                               ↪

↪                          0                                                   ↪
↪                                                                              ↪
↪         2                      2                                             ↪
↪ (θ₄)⋅sin (θ₅) + 0.5758626


Vetor de Coriolis/Centrípeta C(q, qd) [4x1]:



⎡                                                                              ↪
⎢                                                                              ↪
⎢                                                                              ↪
⎢                               2                                              ↪
⎢0.01541109025⋅sin(θ₄ - 2⋅θ₅)⋅θ₄̇  - 0.0308221805⋅sin(θ₄ - 2⋅θ₅)⋅θ₄̇⋅θ₅̇ - 0.0154 ↪
⎢                                                                              ↪
⎢                                                                              ↪
⎢                                                                              ↪
⎢                                                                              ↪
⎢                                                                              ↪
⎣                                                                              ↪

↪                                                                              ↪
↪                       


Vetor de Gravidade/Empuxo G(q) [4x1]:



⎡                                                                    0         ↪
⎢                                                                              ↪
⎢-0.802180069946999⋅sin(θ₃)⋅sin(θ₅) + 6.90343885161901⋅sin(θ₃) - 4.78684930960 ↪
⎢                                                                              ↪
⎢                                   (0.802180069946999⋅sin(θ₄)⋅cos(θ₅) - 4.786 ↪
⎢                                                                              ↪
⎣                              0.802180069946999⋅sin(θ₃)⋅sin(θ₅)⋅cos(θ₄) + 0.8 ↪

↪                                                              ⎤
↪                                                              ⎥
↪ 8⋅sin(θ₄)⋅cos(θ₃) - 0.802180069946999⋅cos(θ₃)⋅cos(θ₄)⋅cos(θ₅)⎥
↪                                                              ⎥
↪ 849309608⋅cos(θ₄))⋅sin(θ₃)                                   ⎥
↪                                                              ⎥
↪ 02180069946999⋅cos(θ₃)⋅cos(θ₅)          